Dataset Link:https://www.kaggle.com/datasets/sankha1998/emotion

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import pandas as pd
angry_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WhatsApp/Emotion(angry).csv')
happy_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WhatsApp/Emotion(happy).csv')
sad_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WhatsApp/Emotion(sad).csv')
combined_df=pd.concat([angry_df,happy_df,sad_df],axis=0)

In [3]:
content_df=combined_df['content']
content_df=content_df.sample(600)

In [4]:
content_df

444        My heart is perfect because.. you are inside.
269          I Am Who I Am. Your Approval Is Not Needed.
282    There’s always a person that you hate for no r...
660    Love begins with a smile, grows with a kiss, a...
343    To be happy does not mean that all is perfect....
                             ...                        
516                     love me less but love me long...
612    We’re the Perfect Couple, We’re just not in th...
62     You left without a reason, so please don't com...
45     There are 3 types of people in the world- vege...
534    ['You Hurt Me But I Still Love You.', 'True Lo...
Name: content, Length: 600, dtype: object

In [5]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer

In [6]:
tokenizer=Tokenizer()

In [7]:
tokenizer.fit_on_texts(content_df)

In [8]:
# len(tokenizer.word_index)

In [9]:
tokenizer.texts_to_sequences(content_df)[0:7]

[[27, 51, 6, 263, 31, 2, 22, 100],
 [5, 70, 32, 5, 70, 16, 953, 6, 24, 906],
 [441, 54, 7, 55, 10, 2, 137, 8, 149, 91],
 [48, 1109, 49, 7, 71, 1110, 49, 7, 907, 20, 278, 49, 7, 1111],
 [3,
  18,
  107,
  868,
  24,
  130,
  10,
  109,
  6,
  263,
  12,
  271,
  10,
  2,
  41,
  954,
  3,
  221,
  472,
  873,
  1112],
 [1113,
  5,
  442,
  41,
  74,
  3,
  137,
  28,
  32,
  137,
  13,
  1114,
  874,
  35,
  443,
  416,
  28,
  32,
  48,
  13],
 [172,
  406,
  5,
  908,
  147,
  428,
  61,
  2,
  20,
  144,
  406,
  5,
  909,
  128,
  5,
  70,
  1115,
  27,
  74]]

In [10]:
input_sequences=[]
for sentence in content_df:
  tokenized_sentence=tokenizer.texts_to_sequences([sentence])[0]
  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[0:i+1])

In [11]:
input_sequences[0:14]

[[27, 51],
 [27, 51, 6],
 [27, 51, 6, 263],
 [27, 51, 6, 263, 31],
 [27, 51, 6, 263, 31, 2],
 [27, 51, 6, 263, 31, 2, 22],
 [27, 51, 6, 263, 31, 2, 22, 100],
 [5, 70],
 [5, 70, 32],
 [5, 70, 32, 5],
 [5, 70, 32, 5, 70],
 [5, 70, 32, 5, 70, 16],
 [5, 70, 32, 5, 70, 16, 953],
 [5, 70, 32, 5, 70, 16, 953, 6]]

In [12]:
max_len=max([len(x) for x in input_sequences])
max_len

2272

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences=pad_sequences(input_sequences,maxlen=max_len,padding='pre')
padded_input_sequences[0:10]

array([[  0,   0,   0, ...,   0,  27,  51],
       [  0,   0,   0, ...,  27,  51,   6],
       [  0,   0,   0, ...,  51,   6, 263],
       ...,
       [  0,   0,   0, ...,   0,   5,  70],
       [  0,   0,   0, ...,   5,  70,  32],
       [  0,   0,   0, ...,  70,  32,   5]], dtype=int32)

In [14]:
X=padded_input_sequences[:,:-1]

In [15]:
y=padded_input_sequences[:,-1]

In [16]:
# One hot encoding
num_classes=len(tokenizer.word_index)+1#num classes mean number of unique words in document
# num_classes  #282
from keras.utils import to_categorical
y = to_categorical(y,num_classes=num_classes)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [19]:
model=Sequential()
model.add(Embedding(num_classes,100,input_length=max_len-1))
model.add(LSTM(150))
model.add(Dense(num_classes,activation='softmax'))

In [20]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2271, 100)         174700    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 1747)              263797    
                                                                 
Total params: 589097 (2.25 MB)
Trainable params: 589097 (2.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.fit(X,y,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
3843/3843 [==============================] - 374s 95ms/step - loss: 4.1370 - accuracy: 0.2300 - val_loss: 2.2059 - val_accuracy: 0.5420
Epoch 2/5
3843/3843 [==============================] - 361s 94ms/step - loss: 1.2448 - accuracy: 0.7726 - val_loss: 0.5912 - val_accuracy: 0.9236
Epoch 3/5
3843/3843 [==============================] - 356s 93ms/step - loss: 0.4231 - accuracy: 0.9425 - val_loss: 0.2801 - val_accuracy: 0.9554
Epoch 4/5
3843/3843 [==============================] - 355s 92ms/step - loss: 0.2617 - accuracy: 0.9546 - val_loss: 0.1947 - val_accuracy: 0.9621
Epoch 5/5
3843/3843 [==============================] - 357s 93ms/step - loss: 0.1998 - accuracy: 0.9598 - val_loss: 0.1545 - val_accuracy: 0.9691


In [26]:
import numpy as np
text = " There are 3 types"

for i in range(15):
  token_text=tokenizer.texts_to_sequences([text])[0]
  padded_token=pad_sequences([token_text],maxlen=2271,padding='pre')
  pos=np.argmax(model.predict(padded_token))
  for word,index in tokenizer.word_index.items():
    if index==pos:
      text=text+" "+word
      print(text)

1/1 [==============================] - 0s 61ms/step
 There are 3 types of
1/1 [==============================] - 0s 88ms/step
 There are 3 types of people
1/1 [==============================] - 0s 49ms/step
 There are 3 types of people in
1/1 [==============================] - 0s 46ms/step
 There are 3 types of people in your
1/1 [==============================] - 0s 45ms/step
 There are 3 types of people in your heart
1/1 [==============================] - 0s 52ms/step
 There are 3 types of people in your heart to
1/1 [==============================] - 0s 48ms/step
 There are 3 types of people in your heart to make
1/1 [==============================] - 0s 48ms/step
 There are 3 types of people in your heart to make your
1/1 [==============================] - 0s 46ms/step
 There are 3 types of people in your heart to make your mouth
1/1 [==============================] - 0s 46ms/step
 There are 3 types of people in your heart to make your mouth work
1/1 [==============================